In [18]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

if api_key is None:
    raise ValueError("GROQ_API_KEY not found in environment variables")

model_client = OpenAIChatCompletionClient(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    # model="meta-llama/llama-4-maverick-17b-128e-instruct",
    model="llama-3.1-8b-instant",
    model_info={
        "family": "llama",
        "vision": False,
        "function_calling": True,
        "json_output": True,
    },
) 

e:\Udemy Courses\Autogen_Crash_Course\.venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [19]:
from autogen_agentchat.agents import AssistantAgent

story_agent = AssistantAgent(
    name="story_agent",
    model_client=model_client,
    system_message="You are a story writer. Write a short story about a dragon and a brave knight."
)

In [20]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content="Write a short story about a dragon and a brave knight in less than 200 words.",
        source='user'
    )
    result = await story_agent.run(task=task)
    response = result.messages[-1].content
    print(response)

await test_simple_agent()

In the land of Eldrador, where mythical creatures roamed free, a brave knight named Sir Edward dared to face the fiercest beast of all – the dragon, Zorvath. Legend had it that Zorvath's fire could melt steel and its roar could shake mountains.

Sir Edward approached the dragon's lair, his armor shining in the fading sunlight. Zorvath awakened, its piercing gaze fixating on the knight. The air was electric with tension as the dragon spread its wings, ready to unleash a fiery blast.

Undaunted, Sir Edward charged forward, his sword flashing in the dim light. Zorvath breathed a stream of flames, but the knight dodged and weaved, avoiding the inferno with ease. Seizing the moment, he struck, his sword biting deep into the dragon's scales.

Zorvath roared in surprise, stumbling backward. Sir Edward pressed his advantage, plunging his sword deeper into the dragon's heart. As the beast fell silent, the land of Eldrador was forever changed, and Sir Edward's name etched into history as the bra

### MultiAgent System

In [21]:
plot_agent = AssistantAgent(
    name="plot_writer",
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey."
)

character_agent = AssistantAgent(
    name="character_writer",
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villain in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name="ending_writer",
    model_client=model_client,
    system_message="You write engaging endings, concluding the story with a twist."
)

In [22]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent],
    max_turns=3 # Number of agent executions
)

In [ ]:
async def test_team():
    team_task = TextMessage(
        content="Write a short story about a brave boy and his Pokemon. Keep it upto 50 words.",
        source="user"
    )
    result = await team.run(task=team_task)
    response = result.messages[-1].content
    print(response)

    print("\n---------Analyzing the responses of individual agents---------\n")

    for each_agent_message in result.messages:
        print(f'{each_agent_message.source}: {each_agent_message.content}')

await test_team()

As the stormy night subsided, and the last rays of sunlight disappeared, Max and Charizard ventured deeper into the uncharted region. The darkness receded, replaced by the faint glow of luminescent mushrooms, guiding them through the dense foliage. Charizard's eyes gleamed with an inner fire, as if anticipating the battles ahead.

Suddenly, a faint rustling emerged from a nearby thicket. Charizard's wings unfolded, its magnificent form poised for battle. A small, timid Emolga peered out, its eyes wide with fear. Max smiled, sensing the Emolga's hesitation.

Charizard approached cautiously, allowing the Emolga to sniff its magnificent form. To Max's surprise, the Emolga's fear dissipated, and it cautiously reached out to Charizard's flame-patterned underbelly. As their eyes met, a spark of understanding flashed between them.

The Emolga revealed that it was once a member of The Shadow Hand's army, but had become disillusioned with Lexi's ambitions and fled. It had been searching for a w

#### Using run_stream function to run the team

In [26]:
from autogen_agentchat.base import TaskResult

await team.reset()
async for message in team.run_stream(task="Write a short poem about the fall season"):
    if isinstance(message, TaskResult):
        print("Stop Reason: ", message.stop_reason)
    else:
        print(f"{message.source}: {message.content}")

user: Write a short poem about the fall season
plot_writer: As leaves begin their wistful dance,
Golden hues and crimson take a stance,
The air is crisp, the winds are cold,
A season's farewell, young and old.

In Alola's islands, wild Pokémon roam,
Witnessing change, their homes now bare of bloom,
But in the midst of autumn's hue,
Their spirits shine, their bond anew.

Pikachu's leaves, a flutter of delight,
As Squirtle plays, beneath the golden light,
Charizard soars, a majestic sight,
In fall's fleeting beauty, pure and bright.

The season's passage, swift and free,
A time for friends, for love, for glee,
So let us cherish, the fall's short reign,
And in the journey, find joy and love that remain.
character_writer: I'd like to develop the Pokémon and the villain character you mentioned earlier.

**The Main Pokémon: Luxara**

Luxara is a unique, hybrid Pokémon, born from the fusion of a Raichu and a Tapu Fini. This Pokémon has an appearance similar to its mother, the Tapu Fini, with 

#### As an agent can be run only once using the run() method, we can create a single agent team that can be run multiple times based on the max_turns parameter value in order to run the same agent multiple times.